# O1v5-SONATA rewiring based on simplified models of connectivity
This is an auxiliary notebook for configuring/testing 1st..5th-order connectome rewiring, which

 - sets up connectome manipulations (rewiring)
 - writes .json config file for running rewiring using sbatch

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.5.dev1 or later; with v4 config format and `parallel-manipulator` entry point)

ℹ️ Related ticket: [[ACCS-49](https://bbpteam.epfl.ch/project/issues/browse/ACCS-49)] "Rewiring V5 connectome based on five simplified models of connectivity"

IMPORTANT: For running actual model building on BB5, use <code>sbatch run_rewiring.sh manip_config__[...].json</code>


In [1]:
# Initialization

""" Global imports """
import json
import os

In [2]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def order_from_name(name):
    order_str = name.split("Order")[0]
    order = [int(i) for i in order_str if i.isdigit()]
    assert len(order) == 1, "ERROR: Order could not be determined!"
    return order[0]

def order_from_model_file(model_file):
    with open(model_file, "r") as f:
        model_dict = json.load(f)
    order = order_from_name(model_dict.get("model", ""))
    assert order > 0, "ERROR: Invalid model order!"
    return order

def add_manip_to_config(manip_config, amount_pct, prob_model_file, delay_model_file, props_model_file, calib_run=True, p_scale=None):
    """Adds a specific n-th order rewiring operation to the (v4) manipulation config dict (in-place)."""
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    order = order_from_model_file(prob_model_file)
    if p_scale is None:
        p_scale = 1.0
    else:
        assert p_scale > 0.0, "ERROR: p_scale out of range!"
    manip_config['manip'] = {'name': f'ConnRewireOrder{order}mc2EE{amount_pct}pct',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'sel_dest': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'gen_method': 'duplicate_randomize', 
                                       'amount_pct': amount_pct,
                                       'estimation_run': calib_run,
                                       'opt_nconn': not calib_run,
                                       'p_scale': p_scale,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel):
    run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


In [4]:
# Rewiring configuration
circuit_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA'
circuit_name = os.path.split(circuit_path)[-1]
# NOT USED # circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config.json') # SONATA config (.json)
# NOT USED # blue_config = os.path.join('..', 'CircuitConfig_TC_BlobStim') # Optional; path rel. to path location of circuit config
circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config_tmp.json') # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]
# blue_config = os.path.join('..', 'CircuitConfig_TC_BlobStim_tmp') # Optional; path rel. to path location of circuit config

output_base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs']  # Write to local and proj9 folder


In [5]:
# Model locations (MC2 column models!!)
models_path = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model')  # Location of model files
prob_model_files = [os.path.join(models_path, f'ConnProb1stOrderO1v5mc2EE.json'),
                    os.path.join(models_path, f'ConnProb2ndOrderComplexO1v5mc2EE.json'),
                    os.path.join(models_path, f'ConnProb3rdOrderComplexO1v5mc2EE.json'),
                    os.path.join(models_path, f'ConnProb4thOrderO1v5mc2EE.json'),
                    os.path.join(models_path, f'ConnProb5thOrderO1v5mc2EE.json')]
delay_model_file = os.path.join(models_path, f'DistDepDelayO1v5mc2EE.json')
props_model_file = os.path.join(models_path, 'ConnPropsPerPathwayO1v5mc2EE.json')


## Create rewiring configs
- Central column (MC2) of O1v5 connectome only *)
- EXC-EXC connections only

*) Otherwise, 1st order probabilities will be much lower within the central column, which is the one we are extracting data from in the end. For consistency, we keep the surrounding columns constant for all simplified models.

In [50]:
# 1st-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[0], delay_model_file, props_model_file, calib_run=False, p_scale=1.0)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-3__'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder1mc2EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder1mc2EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder1mc2EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct" 500


In [44]:
# 2nd-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[1], delay_model_file, props_model_file, calib_run=False, p_scale=1.0064412916711973)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-5__'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder2mc2EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder2mc2EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder2mc2EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct" 500


In [73]:
# 3rd-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[2], delay_model_file, props_model_file, calib_run=False, p_scale=1.0063058207212865)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder3mc2EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder3mc2EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder3mc2EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct" 500


In [20]:
# 4th-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[3], delay_model_file, props_model_file, calib_run=False, p_scale=1.0238483669742555)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-2__'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder4mc2EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder4mc2EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder4mc2EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder4mc2EE100pct" 500


In [34]:
# 5th-order rewiring [CALIBRATION & REWIRING RUNS]
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[4], delay_model_file, props_model_file, calib_run=False, p_scale=1.0423059239398222)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__calib-4__'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnRewireOrder5mc2EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder5mc2EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder5mc2EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct" 500


---
## Matching number of connections

<u>Related ticket</u>: [ACCS-46](https://bbpteam.epfl.ch/project/issues/browse/ACCS-46)

<u>Procedure</u>:
1. Initialize scaling factor p_scale = 1.0
2. Run calibration (evaulation) run with p_scale
3. Check connection counts from data logs and compute new scaling factor p_scale'
4. Adjust scaling factor p_scale = p_scale * p_scale'
5. Repeat steps 2. to 5. until convergence$^1$ (or oscillation)
6. Run actual rewiring with converged p_scale (or closest value, in case of oscillation)

$^1$Convergence ... Number of connections match exactly

### Check connection counts and adjust scaling factor

In [7]:
import tqdm
import numpy as np

In [40]:
# Estimate global probability scaling factor for matching #connections (from log file of estimation or actual rewiring run)
log_type = 'Rewiring' # 'Rewiring'...actual rewiring, 'Estimation'...estimation run
N_split = 500
alt_date = None

# 1st-order rewiring
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct__calib-0__/logs/connectome_manipulation_2023-06-22_13h58.log'  # DIFF = 4450.0 (0.07% rel. to wiring target) => OPTIMUM
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct__calib-1__/logs/connectome_manipulation_2023-06-22_14h06.log'  # DIFF = 4450.0 (0.07% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct__calib-2__/logs/connectome_manipulation_2023-06-22_14h09.log'  # DIFF = -22117.0 (-0.33% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct__calib-3__/logs/connectome_manipulation_2023-06-23_13h49.log'  # DIFF = 4450.0 (0.07% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 0.9993379405726531, 0.9993379405726531, 1.0033035673209574, 0.9993379405726531]
# p_scale_opt = np.prod(p_scales[:1]) = 1.0
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1mc2EE100pct/logs/connectome_manipulation_2023-06-22_14h56.log'  # DIFF = 4450 (0.07% rel. to wiring target) => ACTUAL REWIRING RUN

# 2nd-order rewiring
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-0__/logs/connectome_manipulation_2023-06-22_16h06.log'  # DIFF = -43029.0 (-0.64% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-1__/logs/connectome_manipulation_2023-06-22_16h36.log'  # DIFF = 43.0 (0.00% rel. to wiring target)
# alt_date = '2023-06-22_16h37'  # Alternative date/time, since mismatch (~1s) between time stamps of main log file and data logs possible!! [NEEDS TO BE FIXED IN LOGGING MODULE]
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-2__/logs/connectome_manipulation_2023-06-23_11h50.log'  # DIFF = -3.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-3__/logs/connectome_manipulation_2023-06-23_12h14.log'  # DIFF = 2.0 (0.00% rel. to wiring target) => OPTIMUM
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-4__/logs/connectome_manipulation_2023-06-23_12h23.log'  # DIFF = -2.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct__calib-5__/logs/connectome_manipulation_2023-06-23_12h33.log'  # DIFF = 2.0 (0.00% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 1.0064472850650257, 0.9999935983745231, 1.0000004466280918, 0.9999997022481604, 1.000000297752017, 0.9999997022481604]
# p_scale_opt = np.prod(p_scales[:4]) = 1.0064412916711973
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder2mc2EE100pct/logs/connectome_manipulation_2023-06-23_13h35.log'  # DIFF = 2 (0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 3rd-order rewiring
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct__calib-0__/logs/connectome_manipulation_2023-06-23_15h45.log'  # DIFF = -42057.0 (-0.63% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct__calib-1__/logs/connectome_manipulation_2023-06-23_15h52.log'  # DIFF = -37.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct__calib-2__/logs/connectome_manipulation_2023-06-23_16h09.log'  # DIFF = 3.0 (0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct__calib-3__/logs/connectome_manipulation_2023-06-23_16h20.log'  # DIFF = -2.0 (-0.00% rel. to wiring target) => OPTIMUM
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct__calib-4__/logs/connectome_manipulation_2023-06-23_16h23.log'  # DIFF = 3.0 (0.00% rel. to wiring target) => OSCILLATING
# p_scales = [1.0, 1.0063007270173352, 1.000005508441016, 0.9999995533723071, 1.000000297752017, 0.9999995533723071]
# p_scale_opt = np.prod(p_scales[:4]) = 1.0063058207212865
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder3mc2EE100pct/logs/connectome_manipulation_2023-06-23_16h30.log'  # DIFF = -2 (-0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 4th-order rewiring
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder4mc2EE100pct__calib-0__/logs/connectome_manipulation_2023-06-28_16h28.log'  # DIFF = -156426.0 (-2.33% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder4mc2EE100pct__calib-1__/logs/connectome_manipulation_2023-06-28_16h51.log'  # DIFF = -33.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder4mc2EE100pct__calib-2__/logs/connectome_manipulation_2023-06-28_16h58.log'  # DIFF = 0.0 (0.00% rel. to wiring target) => CONVERGED TO EXACT MATCH
# p_scales = [1.0, 1.0238433369026343, 1.0000049129309534, 1.0]
# p_scale_opt = np.prod(p_scales[:3]) = 1.0238483669742555
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder4mc2EE100pct/logs/connectome_manipulation_2023-06-28_17h08.log'  # DIFF = 0 (0.00% rel. to wiring target) => ACTUAL REWIRING RUN

# 5th-order rewiring
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct__calib-0__/logs/connectome_manipulation_2023-06-28_17h10.log'  # DIFF = -272608.0 (-4.06% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct__calib-1__/logs/connectome_manipulation_2023-06-28_17h16.log'  # DIFF = -22.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct__calib-2__/logs/connectome_manipulation_2023-06-28_17h22.log'  # DIFF = -5.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct__calib-3__/logs/connectome_manipulation_2023-06-28_17h30.log'  # DIFF = -1.0 (-0.00% rel. to wiring target)
# main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct__calib-4__/logs/connectome_manipulation_2023-06-28_17h45.log'  # DIFF = 0.0 (0.00% rel. to wiring target) => CONVERGED TO EXACT MATCH
# p_scales = [1.0, 1.0423015790626053, 1.0000032752819386, 1.000000744380375, 1.0000001488759862, 1.0]
# p_scale_opt = np.prod(p_scales[:5]) = 1.0423059239398222
main_log_file = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder5mc2EE100pct/logs/connectome_manipulation_2023-06-28_17h59.log'  # DIFF = 0 (0.00% rel. to wiring target) => ACTUAL REWIRING RUN
alt_date = '2023-06-28_18h00'  # Alternative date/time, since mismatch (~1s) between time stamps of main log file and data logs possible!! [NEEDS TO BE FIXED IN LOGGING MODULE]

log_path, log_fn = os.path.split(main_log_file)  # Main log path/filename
log_date = os.path.splitext(log_fn)[0].split('connectome_manipulation_')[1]

log_folders = [os.path.join(log_path, d) for d in os.listdir(log_path) if os.path.isdir(os.path.join(log_path, d))]
# data_log_fns = [f'connectome_manipulation.task-{i}_{log_date}.{log_type}Stats_{i + 1}_{N_split}.npz' for i in range(N_split)]
if log_type == 'Rewiring':  # In rewiring run, an ..Indices.. data log must always exist for all splits (so we can check for missing splits)
    data_log_splits = [f'{log_date}.{log_type}Indices_{i + 1}_{N_split}.npz' for i in range(N_split)]
else:  # ... whereas in estimation run, a ..Stats.. data log must always exist for all splits (so we can check for missing splits)
    data_log_splits = [f'{log_date}.{log_type}Stats_{i + 1}_{N_split}.npz' for i in range(N_split)]

if alt_date is not None:
    data_log_splits_alt = [dlog.replace(log_date, alt_date) for dlog in data_log_splits]
else:
    data_log_splits_alt = [None] * len(data_log_splits)

conn_count_orig_all = 0 # Overall input connection count
conn_count_pred_all = 0 # Overall output connection count after rewiring (average prediction)
conn_count_actu_all = 0 # Overall output connection count after rewiring (actual value)
conn_count_orig_sel = 0 # Input connection count within selected rewiring target
conn_count_pred_sel = 0 # Output connection count within selected rewiring target after rewiring (average prediction)
conn_count_actu_sel = 0 # Output connection count within selected rewiring target after rewiring (actual value)
diff_from_avg = []
split_count = 0
for split_name, alt_name in tqdm.tqdm(zip(data_log_splits, data_log_splits_alt), desc="Collecting log files"):
    # Search for log file in all log folders
    folder_name = None
    file_name = None
    for sp_nm in [split_name, alt_name]:
        if sp_nm is None:
            break
        for fld in log_folders:
            tmp_name = [fn for fn in os.listdir(fld) if sp_nm in fn]
            assert len(tmp_name) <= 1, 'ERROR: Multiple files found in current folder!'
            if len(tmp_name) == 1:  # Match
                assert folder_name is None, 'ERROR: Folder not unique!'
                assert file_name is None, 'ERROR: File not unique!'
                folder_name = fld
                file_name = tmp_name[0]
        if folder_name is not None and file_name is not None:
            break
    assert folder_name is not None and file_name is not None, f'ERROR: Log file ..{split_name} {"" if alt_date is None else "(or alternative)"} not found!'

    # Read log file
    file_name = file_name.replace("Indices", "Stats")  # In any case, the data log to load is the ..Stats.. one, if existing
    dlog = os.path.join(folder_name, file_name)
    if not os.path.exists(dlog):
        continue
    split_count += 1
    stats_dict = np.load(dlog)
    conn_count_orig_sel += np.sum(stats_dict['input_conn_count_sel'])
    conn_count_pred_sel += np.sum(stats_dict['output_conn_count_sel_avg'])
    conn_count_orig_all += np.sum(stats_dict.get('input_conn_count', 0))
    conn_count_pred_all += np.sum(stats_dict['output_conn_count_sel_avg']) + np.sum(stats_dict.get('input_conn_count', 0)) - np.sum(stats_dict['input_conn_count_sel'])
    if log_type == 'Rewiring':
        conn_count_actu_sel += np.sum(stats_dict['output_conn_count_sel'])
        assert stats_dict['output_conn_count'] == np.sum(stats_dict['output_conn_count_sel']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
        conn_count_actu_all += stats_dict['output_conn_count']
        diff_from_avg += list(stats_dict['output_conn_count_sel'] - stats_dict['output_conn_count_sel_avg'])

print(f'Non-empty splits: {split_count}/{N_split}\n')

if log_type == 'Rewiring':
    print('<ACTUAL REWIRED INSTANCE>')
    print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_rewired = {conn_count_actu_all}, DIFF = {conn_count_actu_all - conn_count_orig_all} ({100.0 * (conn_count_actu_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
    print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_rewired = {conn_count_actu_sel}, DIFF = {conn_count_actu_sel - conn_count_orig_sel} ({100.0 * (conn_count_actu_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
    print(f'Deviation from average prediction of ingoing #conns of {len(diff_from_avg)} post-neurons: MIN = {np.min(diff_from_avg)}, MAX = {np.max(diff_from_avg)}, #MISMATCH = {np.sum(np.array(diff_from_avg) != 0)}, OVERALL #Conn DIFF = {np.sum(diff_from_avg)}')
    print()

p_scale = conn_count_orig_sel / conn_count_pred_sel
print('<AVERAGE PREDICTION>')
print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_predicted = {conn_count_pred_all}, DIFF = {conn_count_pred_all - conn_count_orig_all} ({100.0 * (conn_count_pred_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_predicted = {conn_count_pred_sel}, DIFF = {conn_count_pred_sel - conn_count_orig_sel} ({100.0 * (conn_count_pred_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
print(f'Probability scaling p_scale = {conn_count_orig_sel} / {conn_count_pred_sel} = {p_scale}')

Non-empty splits: 72/500

<ACTUAL REWIRED INSTANCE>
Overall: #Conns_orig = 14138043, #Conns_rewired = 14138043, DIFF = 0 (0.00%)
Within wiring target: #Conns_orig = 6717001, #Conns_rewired = 6717001, DIFF = 0 (0.00%)
Deviation from average prediction of ingoing #conns of 26567 post-neurons: MIN = 0, MAX = 0, #MISMATCH = 0, OVERALL #Conn DIFF = 0

<AVERAGE PREDICTION>
Overall: #Conns_orig = 14138043, #Conns_predicted = 14138043, DIFF = 0 (0.00%)
Within wiring target: #Conns_orig = 6717001, #Conns_predicted = 6717001, DIFF = 0 (0.00%)
Probability scaling p_scale = 6717001 / 6717001 = 1.0


---
## Tools for post-processing

### Create BlueConfig

In [ ]:
# TODO

### Create bbp-workflow config

In [15]:
# TODO

---

In [1]:
# NEEDS TO BE FIXED:
# -Missing "components" section in generated SONATA config (absoulte paths!!) => So far, added manually